In [1]:
import torch

In [2]:
#devito set up
from abc import ABC, abstractmethod
from devito import Operator, Function
from numpy import array
from devito import Grid, Function, dimensions, Eq, Inc
import sympy
class Layer(ABC):
    def __init__(self, input_data):
        self._input_data = input_data
        self._R = self._allocate()

    @abstractmethod
    def _allocate(self) -> Function:
        # This method should return a Function object corresponding to
        # an output of the layer.
        pass

    def execute(self) -> (Operator, array):
        op = Operator(self.equations())
        op.cfunction

        return (op, self._R.data)

    @abstractmethod
    def equations(self) -> list:
        pass

In [11]:
single_image = torch.rand(0,1,32,32)


In [16]:
class Subsampling_4d(Layer):
    def __init__(self, kernel_size, feature_map, function,
                 stride=(1, 1), padding=(0, 0), activation=None,
                 bias=0):
        # All sizes are expressed as (rows, columns).

        #self._error_check(kernel_size, feature_map, stride, padding)

        self._kernel_size = kernel_size
        self._function = function
        self._activation = activation
        self._bias = bias

        self._stride = stride
        self._padding = padding

        super().__init__(input_data=feature_map)


    def _allocate(self):
        map_height = self._input_data.shape[2] + 2 * self._padding[0]
        map_width = self._input_data.shape[3] + 2 * self._padding[1]
        kernel_height, kernel_width = self._kernel_size
        a, b, c, d = dimensions('a b c d')
        gridB = Grid(shape=(self._input_data.shape[0], self._input_data.shape[1], map_height, map_width),\
                    dimensions=(a, b, c, d))
        B = Function(name='B', grid=gridB, space_order=0)

        e, f, g, h = dimensions('e f g h')
        gridR = Grid(shape=( self._input_data.shape[0],  self._input_data.shape[1],\
                            (map_height - kernel_height + self._stride[0])
                            // self._stride[0],
                            (map_width - kernel_width + self._stride[1])
                            // self._stride[1]),
                     dimensions=(e, f, g, h))
        print(gridR)
        R = Function(name='R', grid=gridR, space_order=0)
        #add padding to start and end of each row
        for image in range(self._input_data.shape[0]):
            for channel in range(self._input_data.shape[1]):
                for i in range(self._padding[0], map_height - self._padding[0]):
                    B.data[image, channel, i] = \
                        np.concatenate(([0] * self._padding[1],
                                        self._input_data[image, channel, i - self._padding[0]],
                                        [0] * self._padding[1]))

        self._B = B
        return R

    def equations(self):
        a, b, c, d = self._B.dimensions
        kernel_height, kernel_width = self._kernel_size
        images = self._input_data.shape[0]
        channels = self._input_data.shape[1] 
        rhs = (self._function([self._B[image, channel, self._stride[0] * c + i,
                                      self._stride[1] * d + j]
                              for i in range(kernel_height)
                              for j in range(kernel_width)])
                              for channel in range(channels)
                            for image in range(images))
        print("rhs:", rhs)
        print("type:", type(rhs))
        if self._activation is not None:
            rhs = self._activation(rhs)

        return [Eq(self._R[a, b, c, d], rhs)]

Sample_obj4 = Subsampling_4d((2,2),single_image, lambda l: sympy.Max(*l))
#A.equations()
tup4 = Sample_obj4.execute()
tup4[0].apply()
tup4[1].shape

Grid[extent=(1.0, 1.0, 1.0, 1.0), shape=(0, 1, 31, 31), dimensions=(e, f, g, h)]
rhs: <generator object Subsampling_4d.equations.<locals>.<genexpr> at 0x7f75fd5255c8>
type: <class 'generator'>


SympifyError: SympifyError: <generator object Subsampling_4d.equations.<locals>.<genexpr> at 0x7f75fd5255c8>